In [ ]:
# ! conda install -c conda-forge ipywidgets

In [3]:
import tensorflow
print(tensorflow.__version__)

2.1.0


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

# Let's chat for 5 lines
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # pretty print last ouput tokens from bot
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

>> User:Hola
DialoGPT: Hola
>> User:Adios
DialoGPT: Hola
>> User:Que haces?
DialoGPT: Hola
>> User:What are you doing?
DialoGPT: Hola
>> User:Ok
DialoGPT: Hola


In [4]:
# Let's chat for 5 lines
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # pretty print last ouput tokens from bot
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

>> User:Hello!
DialoGPT: Hello! :D
>> User:How are you?
DialoGPT: I'm good! How are you?
>> User:Pretty good!
DialoGPT: That's good!
>> User:Can you recommend me a movie?
DialoGPT: I can!
>> User:Tell me
DialoGPT: I will!


In [2]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2):

In [5]:
model2 = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

In [30]:
def jaccard_similarity(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

In [33]:
chat_history_ids = tokenizer.encode('Hello' + tokenizer.eos_token, return_tensors='pt')
reward = 0
for episode in range(3):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
#     new_user_input_ids = tokenizer.encode(tokenizer.eos_token, return_tensors='pt')
    question = model2.generate(chat_history_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, question], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # pretty print last output tokens from bot
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))
    response = chat_history_ids[:, bot_input_ids.shape[-1]:][0]
    history = chat_history_ids[:, :bot_input_ids.shape[-1]][0]
    reward -= jaccard_similarity(response.numpy(), history.numpy())
    print(reward)
    print(response)
    print(history)
#     print(chat_history_ids)
    print(tokenizer.decode(chat_history_ids[0], skip_special_tokens=True))
    


DialoGPT: Hello! :D
-0.5555555555555556
tensor([15496,  5145,  1058,    35, 50256])
tensor([15496, 50256, 15496, 50256, 15496,  5145,  1058,    35, 50256])
HelloHelloHello! :DHello! :D
DialoGPT: Hello! :D
-0.7070707070707071
tensor([15496,  5145,  1058,    35, 50256])
tensor([15496, 50256, 15496, 50256, 15496,  5145,  1058,    35, 50256, 15496,
         5145,  1058,    35, 50256, 15496, 50256, 15496, 50256, 15496,  5145,
         1058,    35, 50256, 15496,  5145,  1058,    35, 50256, 15496,  5145,
         1058,    35, 50256])
HelloHelloHello! :DHello! :DHelloHelloHello! :DHello! :DHello! :DHello! :D
DialoGPT: Hello!
-0.7441077441077442
tensor([15496,  5145, 50256])
tensor([15496, 50256, 15496, 50256, 15496,  5145,  1058,    35, 50256, 15496,
         5145,  1058,    35, 50256, 15496, 50256, 15496, 50256, 15496,  5145,
         1058,    35, 50256, 15496,  5145,  1058,    35, 50256, 15496,  5145,
         1058,    35, 50256, 15496,  5145,  1058,    35, 50256, 15496, 50256,
        15496

In [9]:
tokenizer.decode(chat_history_ids[0], skip_special_tokens=True)

"Hello!Hello! :DHow are you?I'm good! How are you?Pretty good!That's good!Can you recommend me a movie?I can!Tell meI will!"

In [12]:
chat_history_ids

tensor([[15496,     0, 50256, 15496,  5145,  1058,    35, 50256,  2437,   389,
           345,    30, 50256,    40,  1101,   922,  5145,  1374,   389,   345,
          5633, 50256, 35700,   922,     0, 50256,  2504,   338,   922,  5145,
         50256,  6090,   345,  4313,   502,   257,  3807,    30, 50256,    40,
           460,  5145, 50256, 24446,   502, 50256,    40,   481,  5145, 50256]])

In [26]:
! pip install sklearn

  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1321 sha256=d9b12cb485656d6f1756062b7838f488b857ac0cdd2c93aaed565a77d21656fc
  Stored in directory: c:\users\gonre\appdata\local\pip\cache\wheels\46\ef\c3\157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn
